# 🎯 Projenin Amacı

Bu çalışmanın temel amacı, sosyal medya platformlarından biri olan Twitter üzerinde paylaşılan gönderiler aracılığıyla kullanıcıların duygu durumlarını sınıflandırmaktır. Doğal dil işleme (NLP) teknikleri ve makine öğrenmesi algoritmaları kullanılarak, tweet metinleri üzerinden otomatik duygu analizi gerçekleştirilmiştir. Bu analiz sonucunda, her bir tweet’in olumlu (positive), olumsuz (negative) veya nötr (neutral) olarak sınıflandırılması hedeflenmiştir.

### 📦 1. Gerekli Kütüphanelerin Yüklenmesi ve Verinin İçeri Aktarılması

Bu bölümde, proje için gerekli olan Python kütüphaneleri yüklenir ve CSV dosyasından veri seti içeri alınır.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import re
import nltk

# Veriyi yükle
data = pd.read_csv("twitter.csv")
print(data.head())

### 🧽 2. Metin Temizleme İşlemleri

Bu bölümde, tweet metinleri temizlenerek makine öğrenmesi için daha uygun hale getirilir.

In [ ]:
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string

stopword = set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text = " ".join(text)
    return text

# Temizleme işlemini uygulama
data["tweet"] = data["tweet"].apply(clean)

### 🧠 3. Duygu Analizi Skorlarının Hesaplanması

Tweet'ler üzerinden pozitif, negatif ve nötr duygu skorları hesaplanır ve veri setine eklenir.

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sentiments = SentimentIntensityAnalyzer()

data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["tweet"]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["tweet"]]
data["Neutral"]  = [sentiments.polarity_scores(i)["neu"] for i in data["tweet"]]

### 🧾 4. Kullanılacak Sütunların Seçilmesi

Yalnızca analiz için gerekli sütunlar (tweet, pozitif, negatif, nötr) tutulur.

In [ ]:
data = data[["tweet", "Positive", "Negative", "Neutral"]]
print(data.head())

### 📊 5. Genel Duygu Durumunun Belirlenmesi

Tüm tweet'ler için toplam duygu puanları hesaplanır ve hangi duygu öne çıkıyorsa o duygu durumunu temsil eder.

In [ ]:
x = sum(data["Positive"])
y = sum(data["Negative"])
z = sum(data["Neutral"])

def sentiment_score(a, b, c):
    if (a > b) and (a > c):
        print("Positive 😊 ")
    elif (b > a) and (b > c):
        print("Negative 😠 ")
    else:
        print("Neutral 🙂 ")

sentiment_score(x, y, z)

### 📈 6. Duygu Skorlarının Sayısal Olarak Görüntülenmesi

Toplam pozitif, negatif ve nötr duygu puanları terminale yazdırılır.

In [ ]:
print("Positive: ", x)
print("Negative: ", y)
print("Neutral: ", z)